In [1]:
import pandas as pd
from google.cloud import bigquery
from settings import DATA_DIR, RUN_QUERIES
from jupyter_demo.adapters.repository import DataFrameRepository, BigQueryRepository
from jupyter_demo.content.controls.control import TVRadio
from jupyter_demo.content.plots import distribution, high_level, some_detail
from jupyter_demo.options.interactive import set_interactive_opts
from jupyter_demo.options.pandas import set_pandas_opts
from jupyter_demo.options.plot import set_plot_opts
from jupyter_demo.content.sql_queries import tv_radio_station
from IPython.core.display import display
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt

In [2]:
set_pandas_opts()
set_plot_opts()
set_interactive_opts()

### Table of Contents <a name="toc"></a>
* [SQL Query](#sql-query)
* [SQL Execution](#sql-execution)
* [Sample Data](#sample-data)
* [High Level Aggregation](#high-level)
* [Add Some Detail](#more-detail)
* [Distribution Plot](#distribution)
* [See it on a Map](#see-map")


### SQL Query <a name="sql-query"></a>
Check out the radio station distribution in the US.

[back to top](#toc)

In [3]:
print(tv_radio_station.sql_command)


SELECT
    communityState,
    INITCAP(communityCity) as communityCity,
    service,
    status,
    COUNT(DISTINCT stationId) AS number_of_stations
FROM `bigquery-public-data.fcc_political_ads.broadcast_tv_radio_station`
GROUP BY communityState, communityCity, service, status



### SQL Execution <a name="sql-execution"></a>

[back to top](#toc)

In [4]:
df_repository = DataFrameRepository(repository_path=DATA_DIR)
bq_repository = BigQueryRepository(client=bigquery.Client())
filename = 'broadcast_tv_radio_station.fth'

if RUN_QUERIES:
    tv_radio_station_df = bq_repository.get_df(sql_command=tv_radio_station.sql_command)
    df_repository.add(data_frame=tv_radio_station_df, filename=filename)
else:
    tv_radio_station_df = df_repository.get(filename=filename)


Downloading:   0%|          | 0/11148 [00:00<?, ?rows/s]

### Sample Data <a name="sample-data"></a>

[back to top](#toc)

Broadcast tv/radio stations

In [19]:
display(tv_radio_station_df.head(3))

,communityState,communityCity,service,status,number_of_stations
0,MO,St. Louis,Digital TV,LICENSED,6
1,CA,Mecca,FM Station,LICENSED,3
2,PR,Rio Grande,FM Station,LICENSED,1


Latitudes and longitudes for each city and state

In [22]:
geolocation_df = pd.read_csv('../content/data/uscities.csv')
geolocation_df[['city', 'state_id', 'state_name', 'lat','lng']].head(3)

,city,state_id,state_name,lat,lng
0,New York,NY,New York,40.6943,-73.9249
1,Los Angeles,CA,California,34.1139,-118.4068
2,Chicago,IL,Illinois,41.8373,-87.6862


Join the geo data with the stations

In [23]:
geo_tv_radio_station_df = tv_radio_station_df.merge(
    geolocation_df,
    how='inner',
    left_on=['communityCity', 'communityState'],
    right_on=['city', 'state_id']
)

geo_tv_radio_station_df = geo_tv_radio_station_df[['city', 'state_name', 'lat', 'lng', 'service', 'status', 'number_of_stations']]
geo_tv_radio_station_df.head(3)

,city,state_name,lat,lng,service,status,number_of_stations
0,St. Louis,Missouri,38.6358,-90.2451,Digital TV,LICENSED,6
1,St. Louis,Missouri,38.6358,-90.2451,Digital Class A TV,LICENSED,1
2,St. Louis,Missouri,38.6358,-90.2451,FM Station,LICENSED,9


### High Level Aggregation <a name="high-level"></a>

[back to top](#toc)

In [24]:
high_level_df = geo_tv_radio_station_df.groupby(['state_name']).aggregate('number_of_stations').sum()
high_level_df = pd.DataFrame(high_level_df).reset_index()
high_level_df.head(3)

,state_name,number_of_stations
0,Alabama,409
1,Alaska,201
2,Arizona,288


In [25]:
tv_radio = TVRadio(data_frame=high_level_df)
high_level.display(tv_radio=tv_radio, data_frame=high_level_df)


interactive(children=(IntSlider(value=1, description='min_stations', max=1249, min=1), IntSlider(value=624, de…

### Add some more detail <a name="more-detail"></a>

[back to top](#toc)

In [10]:
tv_radio = TVRadio(data_frame=geo_tv_radio_station_df)
some_detail.display(tv_radio=tv_radio, data_frame=geo_tv_radio_station_df)

interactive(children=(Dropdown(description='state_name', options=('Alabama', 'Alaska', 'Arizona', 'Arkansas', …

### Distribution Plot <a name="distribution"></a>

[back to top](#toc)


In [11]:
distribution.display(tv_radio=tv_radio, data_frame=geo_tv_radio_station_df)

interactive(children=(Dropdown(description='service', options=('AM Station', 'Digital Class A TV', 'Digital TV…

### See it on a map <a name="see-map"></a>

[back to top](#toc)

In [12]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
north_america = world.query('continent == "North America"')

In [13]:
geometry = [Point(xy) for xy in zip(geo_tv_radio_station_df['lng'], geo_tv_radio_station_df['lat'])]
geometry[:3]

In [14]:
geometry_df = gpd.GeoDataFrame(
    geo_tv_radio_station_df,
    geometry = geometry
)

geometry_df.head(3)

,city,state_name,lat,lng,service,status,number_of_stations,geometry
0,St. Louis,Missouri,38.6358,-90.2451,Digital TV,LICENSED,6,POINT (-90.24510 38.63580)
1,St. Louis,Missouri,38.6358,-90.2451,Digital Class A TV,LICENSED,1,POINT (-90.24510 38.63580)
2,St. Louis,Missouri,38.6358,-90.2451,FM Station,LICENSED,9,POINT (-90.24510 38.63580)


In [15]:
from ipywidgets import interact_manual, interact
import mplcursors

@interact_manual
def _display(service=tv_radio.service, status=tv_radio.licence_status):
    #TODO: break this out to a different module
    plt.close()

    _geo_tv_radio_station_df = geo_tv_radio_station_df[
            (geo_tv_radio_station_df['service'] == service)
            & (geo_tv_radio_station_df['status'] == status)
        ]

    _geometry = [Point(xy) for xy in zip(_geo_tv_radio_station_df['lng'], _geo_tv_radio_station_df['lat'])]

    _geometry_df  = gpd.GeoDataFrame(
        _geo_tv_radio_station_df,
        geometry = _geometry
    )

    fig, ax = plt.subplots()
    ax.set_title(f"US {status.title()} {service}")

    north_america.plot(
        ax=ax,
        alpha=0.2,
        color="#00296b",
        edgecolor="#faf6f6"
    )

    for i in range(_geometry_df['number_of_stations'].max()):
        _geometry_df[_geometry_df['number_of_stations'] == i].\
            plot(ax=ax,
                 alpha=0.5,
                 markersize=i*10,
                 label=_geometry_df['number_of_stations']
                 )

        mplcursors.cursor(hover=True)
    plt.show()

interactive(children=(Dropdown(description='service', options=('AM Station', 'Digital Class A TV', 'Digital TV…